In [3]:
import numpy as np

In [4]:
def get_ghost_padding(ndim,dim,igs):
    """
    For a given direction, return the number of ghost cells to pad the current direction, and the index slice of the inner array without the ghost cells.

    Parameters
    ----------
    ndim : int
        Number of dimensions for the problem.
    dim : int
        Current dimension to update
    igs : list
        A list of number of ghost cells in all dimensions, e.g. `[2,2,2]` for 2 ghost cells in the x, y, and z directions.

    Returns
    -------
    tuple
        Number of ghost cells in the current dimension at both edges.
    tuple
        Index slice of the inner domain of the array.

    """
    ghost_padding = [(0,0)] * ndim
    ghost_padding[dim] = (igs[dim],igs[dim])

    padded_idx = np.empty((ndim), dtype=object)
    for idim in range(ndim):
        padded_idx[idim] = slice(igs[idim],-igs[idim])
    padded_idx[dim] = slice(None)

    inner_domain = [slice(None)] * ndim
    inner_domain[dim] = slice(igs[dim],-igs[dim])

    return tuple(ghost_padding),  tuple(inner_domain)

In [9]:
ndim = 2
for i in range(ndim):
    x, idx = get_ghost_padding(ndim,i,[2,3,4])
    print(idx)
    print(x)

(slice(2, -2, None), slice(None, None, None))
((2, 2), (0, 0))
(slice(None, None, None), slice(3, -3, None))
((0, 0), (3, 3))
